## imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

## initialize variables

In [13]:
DEVICE: torch.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"{DEVICE = }")

# read input and target data files
input_data = []
target_rank = []

with open('input_data.txt', 'r') as f_input:
    for line in f_input:
        position, release_date = map(float, line.split()) 
        input_data.append([position, release_date])
input_data_tensor = torch.tensor(input_data)
input_data_tensor = input_data_tensor.view(1, -1)

with open('target_ranks.txt', 'r') as f_target:
    target_lines = f_target.readlines()
target_rank = [float(target_line.strip()) for target_line in target_lines]
target_rank_tensor = torch.tensor(target_rank)
target_rank_tensor = target_rank_tensor.view(1, -1)

print("Input data:", input_data_tensor)
print("Target rank:", target_rank_tensor)

DEVICE = device(type='cpu')
Input data: tensor([[1.0000e+00, 2.0200e+03, 2.0000e+00, 2.0180e+03, 3.0000e+00, 2.0190e+03,
         4.0000e+00, 2.0140e+03, 5.0000e+00, 2.0170e+03, 6.0000e+00, 2.0230e+03,
         7.0000e+00, 2.0200e+03, 8.0000e+00, 2.0060e+03, 9.0000e+00, 2.0200e+03,
         1.0000e+01, 2.0230e+03, 1.1000e+01, 1.9860e+03, 1.2000e+01, 2.0050e+03,
         1.3000e+01, 1.9770e+03, 1.4000e+01, 2.0190e+03, 1.5000e+01, 2.0190e+03,
         1.6000e+01, 2.0150e+03, 1.7000e+01, 2.0150e+03, 1.8000e+01, 2.0190e+03,
         1.9000e+01, 1.9690e+03, 1.0000e+00, 2.0000e+03, 2.0000e+00, 1.9730e+03,
         3.0000e+00, 2.0210e+03, 4.0000e+00, 2.0080e+03, 5.0000e+00, 2.0240e+03,
         6.0000e+00, 2.0110e+03, 7.0000e+00, 2.0220e+03, 8.0000e+00, 1.9650e+03,
         9.0000e+00, 2.0210e+03, 1.0000e+01, 2.0140e+03, 1.1000e+01, 2.0040e+03,
         1.2000e+01, 2.0190e+03, 1.3000e+01, 2.0210e+03, 1.4000e+01, 2.0170e+03,
         1.5000e+01, 2.0240e+03, 1.6000e+01, 2.0100e+03, 1.7000e+01, 

## simple neural network

In [14]:
class LeagueNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int) -> None:
        super(LeagueNN, self).__init__()
        # affine transformations
        self.lin1 = nn.Linear(input_size, hidden_size) 
        # with nonlinearities in between
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(hidden_size, output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # apply things in sequence
        out = self.lin1(x.flatten(start_dim=1))
        out = self.relu(out)
        out = self.lin2(out)
        return out


## test model

In [19]:
# instantiate
model = LeagueNN(
    input_size = 190 * 2,
    hidden_size = 10,
    output_size = 190
)

# loss
loss_criterion = nn.MSELoss() 
optimizer = optim.SGD(model.parameters(), lr=0.01)

epochs = 100000
for epoch in range(epochs):
    # Forward pass
    outputs = model(input_data_tensor)
    
    # Compute the loss
    loss = loss_criterion(outputs, target_rank_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()  # Zero gradients before backward pass
    loss.backward()        # Compute gradients
    optimizer.step()       # Update the weights
    
    if (epoch + 1) % 100 == 0:  # Print loss every 100 epochs
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Test the trained model
with torch.no_grad():  # No need to compute gradients during inference
    predicted = model(input_data_tensor)
    print("Predictions after training:", predicted)
    print("Actual target:", target_rank_tensor)

## TO DO: normalize input and output 

Epoch [100/100000], Loss: 122599.0625
Epoch [200/100000], Loss: 120044.8750
Epoch [300/100000], Loss: 117543.8984
Epoch [400/100000], Loss: 115095.0234
Epoch [500/100000], Loss: 112697.1797
Epoch [600/100000], Loss: 110349.2812
Epoch [700/100000], Loss: 108050.3047
Epoch [800/100000], Loss: 105799.2344
Epoch [900/100000], Loss: 103595.0312
Epoch [1000/100000], Loss: 101436.7656
Epoch [1100/100000], Loss: 99323.4766
Epoch [1200/100000], Loss: 97254.2031
Epoch [1300/100000], Loss: 95228.0391
Epoch [1400/100000], Loss: 93244.0859
Epoch [1500/100000], Loss: 91301.4844
Epoch [1600/100000], Loss: 89399.3359
Epoch [1700/100000], Loss: 87536.8203
Epoch [1800/100000], Loss: 85713.1016
Epoch [1900/100000], Loss: 83927.3828
Epoch [2000/100000], Loss: 82178.8750
Epoch [2100/100000], Loss: 80466.7891
Epoch [2200/100000], Loss: 78790.3594
Epoch [2300/100000], Loss: 77148.8828
Epoch [2400/100000], Loss: 75541.5781
Epoch [2500/100000], Loss: 73967.7734
Epoch [2600/100000], Loss: 72426.7500
Epoch [2700